## 參考   

1.  [文本情感分类（三）：分词 OR 不分词]( https://kexue.fm/archives/3863), LSTM model 但是跑出來不像 blog 所述
2.  [当Bert遇上Keras：这可能是Bert最简单的打开姿势]( https://kexue.fm/archives/6736)

1.   keras-bert：https://github.com/CyberZHG/keras-bert

1.   更多例子所在Github：https://github.com/bojone/bert_in_keras/






from google.colab import drive
drive.mount('/content/drive')

root = 'drive/My Drive/'
!ls

colabok= root + 'ColabOK/'
!pip install keras_bert

In [1]:
#! -*- coding:utf-8 -*-

import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs


maxlen = 100
config_path ='cbert1/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'cbert1/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'cbert1/chinese_L-12_H-768_A-12/vocab.txt'


token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)


Using TensorFlow backend.


In [2]:
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam


bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(1, activation='sigmoid')(x)

model = Model([x1_in, x2_in], p)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=['accuracy']
)
h5_path = 'cbert1/05_02F2.h5'
model.load_weights(h5_path)
model._make_predict_function()  # amazing  https://github.com/keras-team/keras/issues/6462

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
def sentence_inference(x):
  test = x
  maxlen =100
  d=[test,0]
  test=d[0][:maxlen]
  x1, x2 = tokenizer.encode(first=str(test))
  x1=[x1]
  x2=[x2]
  test =[x1, x2]
  model.predict(test)
  if model.predict(test) >=0.5:
    y = 1
  else: y=0
  #print(f'{x} \n {y}')
  return y, model.predict(test)




#!pip install line-bot-sdk

from flask import Flask, request, abort
import json

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

app = Flask(__name__)

#token
line_bot_api = LineBotApi('4UMZ57VIP0FSCa62WsYUUHwQ/KlZBNcyNTEzuOlyAfaIOMPjGSJKyAf4P6KDg8d1kHwImrxNg3fcXvIjxpioTu8OkQKbfI0lsiVcQ9BzkcLt0VkrRLfwgvSsS9wzAxnn48ZPjXHGXUYwUgWrPukIbgdB04t89/1O/w1cDnyilFU=')
#secret
handler = WebhookHandler('182080c576d83b01a786616b8abd1b33')

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    data = json.loads(body)
    y, s = sentence_inference(data['events'][0]['message']['text'])
    if y == 1:
        y = "正面評價"
    else: y = "負評"
    
    app.logger.info("Request body: " + body)
    try:
        line_bot_api.reply_message(data['events'][0]['replyToken'], TextSendMessage(text=y))
        #print('2')

    except InvalidSignatureError:
        #print('pre_3')
        abort(400)
        #print('3')

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print('2_call')
    print(event)
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


if __name__ == "__main__":
    #app.run()
    app.run(host='0.0.0.0', port=8080, threaded=True)
    
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2019 01:57:36] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 01:57:44] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 01:59:30] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 01:59:57] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 02:00:19] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 02:00:30] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 02:00:40] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 02:00:54] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2019 02:01:00] "POST /callback HTTP/1.1" 200 -


# 底下是研究過程中對 Input format 的研究
## 研究  Inference

#


test = ' 作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？为什么在文化上有着深刻纽带关系的中国和日本却在经济发展上有着极大的差异？为什么英国的北美殖民地造就了经济强大的美国，而西班牙的北美殖民却造就了范后的墨西哥？……很有价值，但不包括【中国近代史专业】。螢幕閱讀器支援功能已啟用'
len(test) #160
maxlen #100
d = [test,0]
test = d[0][:maxlen]
#test = test[:maxlen]
#test =[test]
print(test)
print(f'len of test = {len(test)}')
#print(f'test[0][0] is {test[3]}')
#test = test[:100]
x1,x2=tokenizer.encode(first=str(test))
#X1=seq_padding(x1)
#X2=seq_padding(x2)
x1=[x1]
x2=[x2]
test=[x1,x2]
print(type(x1))
print(type(test))  #str
print(len(test))   #2
#maxlen #100
print(len(x1))  #102
print(len(x2))  #102
#x2  # all "0"


# 製作 function

# 測試句子

def sentence_inference(x):
  test = x
  maxlen =100
  d=[test,0]
  test=d[0][:maxlen]
  x1, x2 = tokenizer.encode(first=str(test))
  x1=[x1]
  x2=[x2]
  test =[x1, x2]
  model.predict(test)
  if model.predict(test) >=0.5:
    y = 1
  else: y=0
  print(f'{x} \n {y}')
  return y, model.predict(test)




In [4]:
f_neg = 'cbert1/neg.xls'
f_pos = "cbert1/pos.xls"
neg = pd.read_excel(f_neg, header=None)
pos = pd.read_excel(f_pos, header=None)
pos.head()

,0
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


In [5]:
def sentence_inference(x):
  test = x
  maxlen =100
  d=[test,0]
  test=d[0][:maxlen]
  x1, x2 = tokenizer.encode(first=str(test))
  x1=[x1]
  x2=[x2]
  test =[x1, x2]
  model.predict(test)
  if model.predict(test) >=0.5:
    y = 1
  else: y=0
  print(f'{x} \n {y}')
  return y, model.predict(test)

#model.predict(input_transform('作者很棒, 我很贊同,所列舉的方法很實用'))
sentence_inference('作者很棒, 我很贊同,沒有列舉的方法很實用')
sentence_inference('作者很棒,列舉的方法很實用ㄝ')
sentence_inference('作者很棒, 我很贊同,列舉的方法很實用ㄝ')
sentence_inference('作者很棒, 我很贊同,沒有列舉的方法很實用')
sentence_inference('作者很棒,列舉的方法很實用ㄝ')
y, s=sentence_inference('作者很棒, 我很贊同,列舉的方法很實用ㄝ')
print(y, s)
sentence_inference(pos.iloc[3,0])
sentence_inference(neg.iloc[7833,0])
sentence_inference(pos.iloc[8383,0])
sentence_inference(neg.iloc[173,0])


作者很棒, 我很贊同,沒有列舉的方法很實用 
 0
作者很棒,列舉的方法很實用ㄝ 
 1
作者很棒, 我很贊同,列舉的方法很實用ㄝ 
 0
作者很棒, 我很贊同,沒有列舉的方法很實用 
 0
作者很棒,列舉的方法很實用ㄝ 
 1
作者很棒, 我很贊同,列舉的方法很實用ㄝ 
 0
0 [[0.3782348]]
作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延续，没有战后的民发反思，没有～，就不会让日本成为一个经济强国．当然，美国人也给日本人带来了耻辱．对日中关系也造成了深远的影响．文中揭露了＂东京审判＂中很多鲜为人知的东西．让人惊醒．唉！中国人民对日本的了解是不是太少了． 
 1
装XP非常麻烦，需要准备外置软驱，给光驱及南桥装驱动。官方没有驱动下载。只有打客服电话要他们发给你。尤其声卡，极其麻烦。建议菜鸟要装XP不要买这机子。装Vista的没问题。 
 0
王蜀黍：今天有幸能参加蒙牛举办的三人篮球赛，“客队”第一个亮相，经过一天的比赛，终于闯进了6强，等待我们的是明天与其他学校的对决，加油！——同意：11人 http:t.cn/zjh516R （分享自 @深大小评论） 
 1
这本书好畅销啊,虽然平时不大喜欢看这方面内容的书,但在当当上看到这么多推荐读它的评论,也忍不住买一本来读读看了 
 0


(0, array([[0.00019237]], dtype=float32))

##### TRY

# Line

##### TRY